<a href="https://colab.research.google.com/github/WorldEconomist/Projects/blob/main/Machine%20Learning/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1UbI0LQS-rdzEn8xjbz6OQYlxMLrfnwR8
!gdown 1_0IAa097iLdobdgoawP7qvJfhy1VO_fv

Downloading...
From: https://drive.google.com/uc?id=1UbI0LQS-rdzEn8xjbz6OQYlxMLrfnwR8
To: /content/hh_dataset_x_test.csv
100% 44.3M/44.3M [00:00<00:00, 218MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1_0IAa097iLdobdgoawP7qvJfhy1VO_fv
From (redirected): https://drive.google.com/uc?id=1_0IAa097iLdobdgoawP7qvJfhy1VO_fv&confirm=t&uuid=a22c87a6-b540-4e67-ba56-d39121bcff04
To: /content/hh_dataset_train.csv
100% 177M/177M [00:02<00:00, 87.6MB/s]


Набор данных содержит информацию о вакансиях и соответствующих размерах заработной платы. Задача — прогнозирование размера ЗП.

В качестве алгоритма использовать линейную регресиию.

In [ ]:
!pip install phik

import pandas as pd
import numpy as np
import phik
import json

import nltk
nltk.download('all')
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [3]:
data_raw = pd.read_csv('hh_dataset_train.csv')
X_test = pd.read_csv('hh_dataset_x_test.csv')

#EDA and Preprocessing

Let's look at our data

In [4]:
data_raw.head(1)

,premium,name,department,has_test,response_letter_required,area,type,address,response_url,sort_point_distance,...,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,immediate_redirect_url,mean_salary
0,False,Агент по недвижимости,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Невский...",NaN,NaN,...,[],[],False,"<p>Мы ищем людей, которым не достаточно того, ...","{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,NaN,125000.0


In [5]:
X_test.head(1)

,Id,premium,name,department,has_test,response_letter_required,area,type,address,response_url,...,working_days,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,immediate_redirect_url
0,0,False,Помощник маркетолога,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,...,[],[],[],False,<p>В IT-компанию Ищем Интернет-маркетолога (уд...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Контекстная реклама'}, {'name': 'Ин...","[{'id': '3.206', 'name': 'Печатная реклама', '...",Москва,NaN


Seems like columns do not match

In [6]:
data_raw.shape, X_test.shape

((30072, 32), (7518, 32))

In [7]:
def compare_columns(df1, df2):
    return {
        "only_in_df1": set(df1.columns) - set(df2.columns),
        "only_in_df2": set(df2.columns) - set(df1.columns)
    }

In [8]:
compare_columns(data_raw, X_test)

{'only_in_df1': {'mean_salary'}, 'only_in_df2': {'Id'}}

In [9]:
X_test['Id'].iloc[[0, -1]]

,Id
0,0
7517,7517


This might just be a copy paste index column. Let's check check if this data by any chance were taken out of the original dataset. If not, we can get rid of 'id' column.

In [10]:
sum(data_raw['url'].isin(X_test['url']))

38

Roughly speaking, these datasets are completely different

In [11]:
X_test = X_test.drop(columns = ['Id'])

In [12]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7518 entries, 0 to 7517
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   premium                   7518 non-null   bool   
 1   name                      7518 non-null   object 
 2   department                187 non-null    object 
 3   has_test                  7518 non-null   bool   
 4   response_letter_required  7518 non-null   bool   
 5   area                      7518 non-null   object 
 6   type                      7518 non-null   object 
 7   address                   4753 non-null   object 
 8   response_url              1 non-null      object 
 9   sort_point_distance       0 non-null      float64
 10  published_at              7518 non-null   object 
 11  created_at                7518 non-null   object 
 12  archived                  7518 non-null   bool   
 13  insider_interview         23 non-null     object 
 14  url     

In [13]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30072 entries, 0 to 30071
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   premium                   30072 non-null  bool   
 1   name                      30072 non-null  object 
 2   department                752 non-null    object 
 3   has_test                  30072 non-null  bool   
 4   response_letter_required  30072 non-null  bool   
 5   area                      30072 non-null  object 
 6   type                      30072 non-null  object 
 7   address                   19105 non-null  object 
 8   response_url              0 non-null      float64
 9   sort_point_distance       0 non-null      float64
 10  published_at              30072 non-null  object 
 11  created_at                30072 non-null  object 
 12  archived                  30072 non-null  bool   
 13  insider_interview         94 non-null     object 
 14  url   

Let's check our binary data

In [14]:
for column in data_raw.select_dtypes(include=['bool']).columns:
    print(f"Values in: {column}")
    print(data_raw[column].value_counts(normalize=True))
    print("---"*50)

Values in: premium
premium
False    0.996608
True     0.003392
Name: proportion, dtype: float64
------------------------------------------------------------------------------------------------------------------------------------------------------
Values in: has_test
has_test
False    0.985967
True     0.014033
Name: proportion, dtype: float64
------------------------------------------------------------------------------------------------------------------------------------------------------
Values in: response_letter_required
response_letter_required
False    0.98354
True     0.01646
Name: proportion, dtype: float64
------------------------------------------------------------------------------------------------------------------------------------------------------
Values in: archived
archived
False    0.999867
True     0.000133
Name: proportion, dtype: float64
-------------------------------------------------------------------------------------------------------------------------------

In [15]:
data_raw['mean_salary'].corr(data_raw['accept_temporary'])

0.044610011586470245

Binary data are irrelevant, we can get rid of them.

## Creating a Class for working with both datasets simultaneously

In [16]:
class DataPreprocessor:
    def __init__(self, *datasets):
        self.datasets = list(datasets)

    def apply(self, func):
        for i, df in enumerate(self.datasets):
            self.datasets[i] = func(df)

    def get_datasets(self):
        return tuple(self.datasets)

    def drop_null_columns(self, additional_columns=None):
      for i, df in enumerate(self.datasets):
        null_columns = df.isnull().mean()
        columns_to_drop = null_columns[null_columns > 0.7].index.tolist()
        if additional_columns:
          columns_to_drop.extend(additional_columns)
          self.datasets[i] = df.drop(columns=columns_to_drop)


    def convert_json_string_to_dict(self, string):
        if isinstance(string, str):
            try:
                valid_json_string = string.replace("'", '"')
                return json.loads(valid_json_string)
            except json.JSONDecodeError:
                return string
        return string

    def process_json_strings(self, columns = None):
        for i, df in enumerate(self.datasets):
            if columns is None:
                columns = df.columns
            for column in columns:
                if df[column].dtype == 'object':
                    df[column] = df[column].apply(self.convert_json_string_to_dict)

            self.datasets[i] = df


    def unpack_dict(self, old_col, key, new_col, numeric = False):
      for i, df in enumerate(self.datasets):

        def extract_value(x):
            if isinstance(x, dict):
                return x.get(key)
            elif isinstance(x, list) and all(isinstance(item, dict) for item in x):
                return [item.get(key) for item in x]
            return None

        df[new_col] = df[old_col].apply(extract_value)
        if numeric:
            df[new_col] = pd.to_numeric(df[new_col], errors='coerce')

        self.datasets[i] = df


In [17]:
additional_columns = (
    data_raw.select_dtypes(include = ['bool']).columns.tolist()
    + ['type', 'url', 'address', 'published_at', 'created_at', 'alternate_url',
       'relations', 'contacts', 'working_days', 'working_time_intervals',
       'working_time_modes', 'region', 'employer', 'description', 'name', 'snippet']
)

json_cols = ['area', 'schedule','experience',
             'key_skills', 'specializations']

In [18]:
processor = DataPreprocessor(data_raw, X_test)

In [19]:
processor.drop_null_columns(additional_columns = additional_columns)
processor.process_json_strings(columns = json_cols)

data_new, X_test_new = processor.get_datasets()

In [20]:
processor = DataPreprocessor(data_new, X_test_new)

In [21]:
processor.unpack_dict('area', 'id', 'area_id', numeric = True)
processor.unpack_dict('area', 'name', 'area_name', numeric = False)
processor.unpack_dict('schedule', 'id', 'working_hours', numeric = False)
processor.unpack_dict('key_skills', 'name', 'key_skills_name', numeric = False)
#processor.unpack_dict('specializations', 'name', 'specializations_name', numeric = False)
processor.unpack_dict('specializations', 'profarea_name', 'specializations_area_name', numeric = False)
processor.unpack_dict('experience', 'id', 'experience_req', numeric = False)

data_new, X_test_new = processor.get_datasets()

In [22]:
data_new.head(1)

,area,schedule,experience,key_skills,specializations,mean_salary,area_id,area_name,working_hours,key_skills_name,specializations_area_name,experience_req
0,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'flexible', 'name': 'Гибкий график'}","{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",125000.0,2,Санкт-Петербург,flexible,"[Жилая недвижимость, Активные продажи, Подбор ...","[Строительство, недвижимость, Строительство, н...",noExperience


In [23]:
keys_per_column = {}

for column in data_new.columns:
    keys = set()
    for row in data_new[column]:
        if isinstance(row, dict):
            keys.update(row.keys())
        elif isinstance(row, list):
            for item in row:
                if isinstance(item, dict):
                    keys.update(item.keys())

    if keys:
        keys_per_column[column] = keys

for column, keys in keys_per_column.items():
    print(f"Keys in '{column}': {keys}")

Keys in 'area': {'url', 'id', 'name'}
Keys in 'schedule': {'id', 'name'}
Keys in 'experience': {'id', 'name'}
Keys in 'key_skills': {'name'}
Keys in 'specializations': {'id', 'profarea_id', 'profarea_name', 'name'}


In [24]:
data_new.working_hours.value_counts(normalize = True)

,proportion
working_hours,
fullDay,0.755221
remote,0.151769
flexible,0.064645
shift,0.025239
flyInFlyOut,0.003126


In [25]:
data_new.experience_req.value_counts(normalize = True)

,proportion
experience_req,
between1And3,0.490756
noExperience,0.278332
between3And6,0.208965
moreThan6,0.021947


In [26]:
len(data_new['area_id'].unique())

784

In [27]:
top_10_area = data_new[['area_id', 'area_name']].value_counts().head(10).reset_index()

In [28]:
data_new['global_area'] = data_new['area_id'].apply(lambda x: 'Top_10'
                                                     if x in top_10_area['area_id'].values
                                                     else 'Other')

data_new = data_new.drop(columns = ['area', 'area_id', 'area_name', 'schedule',
                                    'experience', 'key_skills', 'specializations'])

X_test_new['global_area'] = X_test_new['area_id'].apply(lambda x: 'Top_10'
                                                     if x in top_10_area['area_id'].values
                                                     else 'Other')

X_test_new = X_test_new.drop(columns = ['area', 'area_id', 'area_name',
                                        'schedule', 'key_skills',
                                        'specializations', 'experience'])

In [29]:
data_new.global_area.value_counts(normalize = True)

,proportion
global_area,
Top_10,0.507549
Other,0.492451


In [30]:
data_new.head(5)

,mean_salary,working_hours,key_skills_name,specializations_area_name,experience_req,global_area
0,125000.0,flexible,"[Жилая недвижимость, Активные продажи, Подбор ...","[Строительство, недвижимость, Строительство, н...",noExperience,Top_10
1,50000.0,flyInFlyOut,[],"[Информационные технологии, интернет, телеком,...",between1And3,Other
2,80000.0,fullDay,"[Linux, Bash, C/C++]","[Информационные технологии, интернет, телеком,...",between1And3,Top_10
3,120000.0,fullDay,"[1С программирование, Обновление конфигурации ...","[Маркетинг, реклама, PR, Информационные технол...",between1And3,Top_10
4,15000.0,fullDay,"[Составление отчетности, Работа с оргтехникой,...","[Производство, сельское хозяйство]",between1And3,Other


In [31]:
def is_empty_list(x):
    return isinstance(x, list) and len(x) == 0

In [32]:
data_new.map(is_empty_list).any()

,0
mean_salary,False
working_hours,False
key_skills_name,True
specializations_area_name,False
experience_req,False
global_area,False


In [33]:
len(data_new[data_new.map(is_empty_list).any(axis=1)])

5206

In [34]:
data_new = data_new.drop(columns = ['key_skills_name'])
X_test_new = X_test_new.drop(columns = ['key_skills_name'])

In [35]:
data_new[data_new.isna().any(axis=1)]

,mean_salary,working_hours,specializations_area_name,experience_req,global_area


In [36]:
X_test_new[X_test_new.isna().any(axis=1)]

,working_hours,specializations_area_name,experience_req,global_area


In [37]:
data_new.head(5)

,mean_salary,working_hours,specializations_area_name,experience_req,global_area
0,125000.0,flexible,"[Строительство, недвижимость, Строительство, н...",noExperience,Top_10
1,50000.0,flyInFlyOut,"[Информационные технологии, интернет, телеком,...",between1And3,Other
2,80000.0,fullDay,"[Информационные технологии, интернет, телеком,...",between1And3,Top_10
3,120000.0,fullDay,"[Маркетинг, реклама, PR, Информационные технол...",between1And3,Top_10
4,15000.0,fullDay,"[Производство, сельское хозяйство]",between1And3,Other


In [38]:
X_test_new.head(1)

,working_hours,specializations_area_name,experience_req,global_area
0,remote,"[Маркетинг, реклама, PR, Маркетинг, реклама, P...",between1And3,Top_10


specializations_area_name labels are not so disctinct, so we will use MultiLabelBinarizer

In [39]:
phik_overview = data_new[['mean_salary',
                          'working_hours',
                          'experience_req',
                          'global_area']].phik_matrix()
phik_overview.round(2)

interval columns not set, guessing: ['mean_salary']


,mean_salary,working_hours,experience_req,global_area
mean_salary,1.00,0.09,0.16,0.18
working_hours,0.09,1.00,0.13,0.03
experience_req,0.16,0.13,1.00,0.29
global_area,0.18,0.03,0.29,1.00


# Text Preprocessing

In [40]:
lemmatizer = nltk.WordNetLemmatizer()

In [41]:
def process_and_binarize(df, mlb=None):
    df['specializations_area_name_lemma'] = (
        df['specializations_area_name'].apply(
            lambda x: [lemmatizer.lemmatize(word) for word in x]
        )
    )

    if mlb is None:
        mlb = MultiLabelBinarizer()
        encoded = mlb.fit_transform(df['specializations_area_name_lemma'])
    else:
        encoded = mlb.transform(df['specializations_area_name_lemma'])

    encoded_df = pd.DataFrame(encoded, columns=mlb.classes_)

    df = df.drop(columns=['specializations_area_name_lemma', 'specializations_area_name'])

    return pd.concat([df, encoded_df], axis=1), mlb

In [42]:
data_new_encoded, mlb = process_and_binarize(data_new)
X_test_new_encoded, _ = process_and_binarize(X_test_new, mlb=mlb)

In [43]:
def process_and_ohe(df, encoder=None):
    ohe_columns = ['working_hours', 'experience_req', 'global_area']

    if encoder is None:
        encoder = OneHotEncoder(sparse_output=False)
        encoded = encoder.fit_transform(df[ohe_columns])
    else:
        encoded = encoder.transform(df[ohe_columns])

    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(ohe_columns))

    df = df.drop(columns=ohe_columns)

    return pd.concat([df, encoded_df], axis=1), encoder

In [44]:
data_final, encoder = process_and_ohe(data_new_encoded)
X_test_final, _ = process_and_ohe(X_test_new_encoded, encoder=encoder)

In [45]:
data_final.head()

,mean_salary,Автомобильный бизнес,Административный персонал,"Банки, инвестиции, лизинг",Безопасность,"Бухгалтерия, управленческий учет, финансы предприятия",Высший менеджмент,"Государственная служба, некоммерческие организации",Добыча сырья,Домашний персонал,...,working_hours_flyInFlyOut,working_hours_fullDay,working_hours_remote,working_hours_shift,experience_req_between1And3,experience_req_between3And6,experience_req_moreThan6,experience_req_noExperience,global_area_Other,global_area_Top_10
0,125000.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,50000.0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,80000.0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,120000.0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,15000.0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [46]:
X_test_final.head()

,Автомобильный бизнес,Административный персонал,"Банки, инвестиции, лизинг",Безопасность,"Бухгалтерия, управленческий учет, финансы предприятия",Высший менеджмент,"Государственная служба, некоммерческие организации",Добыча сырья,Домашний персонал,Закупки,...,working_hours_flyInFlyOut,working_hours_fullDay,working_hours_remote,working_hours_shift,experience_req_between1And3,experience_req_between3And6,experience_req_moreThan6,experience_req_noExperience,global_area_Other,global_area_Top_10
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [47]:
y = data_final['mean_salary']
X = data_final.drop(columns = ['mean_salary'])

In [48]:
X_train, _, y_train, y_test = train_test_split(X, y, test_size=X_test_final.shape[0], random_state=42)

In [49]:
reg = LinearRegression().fit(X_train, y_train)

In [50]:
y_pred = reg.predict(X_test_final)

In [51]:
mean_absolute_error(y_test, y_pred)

73112.71144133061